In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import random

In [ ]:
data_dir = '../input/car-damage-assessment/image'

In [ ]:
labels=pd.read_csv('../input/car-damage-assessment/data.csv')
labels

In [ ]:
Name=labels.classes.unique().tolist()
print(Name)
print(len(Name))

In [ ]:
N=list(range(len(Name)))
normal_mapping=dict(zip(Name,N)) 
reverse_mapping=dict(zip(N,Name)) 

In [ ]:
path0='../input/car-damage-assessment/image/1436.jpeg'
image=cv2.imread(path0)
print(image.shape)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

In [ ]:
image2=image[40:180,40:180]
print(image2.shape)
plt.imshow(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB))

In [ ]:
image3=cv2.resize(image2,dsize=(50,50),interpolation=cv2.INTER_CUBIC)
print(image3.shape)
plt.imshow(cv2.cvtColor(image3, cv2.COLOR_BGR2RGB))

In [ ]:
labels[labels['image']=='image/'+'1436.jpeg']['classes'].tolist()[0]

In [ ]:
dataset=[]
dataname0=[]
for im in os.listdir(data_dir):
    image=cv2.imread(os.path.join(data_dir,im))
    image2=image[40:180,40:180]
    image3=cv2.resize(image2,dsize=(50,50),interpolation=cv2.INTER_CUBIC)
    dataset+=[image3]
    class0=labels[labels['image']=='image/'+im]['classes'].tolist()[0]
    dataname0+=[class0]

In [ ]:
dataname1=pd.Series(dataname0).map(normal_mapping)

In [ ]:
data=np.array(dataset)
dataname=np.array(dataname1)

In [ ]:
m=len(data)
M=list(range(m))
random.seed(2021)
random.shuffle(M)

In [ ]:
trainX=data[M[0:(m//4)*3]]
trainY0=dataname[M[0:(m//4)*3]]

testX=data[M[(m//4)*3:]]
testY0=dataname[M[(m//4)*3:]]

In [ ]:
labels1=to_categorical(trainY0)
trainY=np.array(labels1)

In [ ]:
trainx,testx,trainy,testy=train_test_split(trainX,trainY,test_size=0.2,random_state=44)

In [ ]:
print(trainx.shape)
print(testx.shape)
print(trainy.shape)
print(testy.shape)

In [ ]:
datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rotation_range=20,zoom_range=0.2,
                        width_shift_range=0.2,height_shift_range=0.2,shear_range=0.1,fill_mode="nearest")

In [ ]:
pretrained_model3 = tf.keras.applications.DenseNet201(input_shape=(50,50,3), include_top=False, weights='/kaggle/input/densenet201/densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5', pooling='avg')


In [ ]:
inputs3 = pretrained_model3.input
x3 = tf.keras.layers.Dense(128, activation='relu')(pretrained_model3.output)
outputs3 = tf.keras.layers.Dense(len(Name), activation='softmax')(x3)
model = tf.keras.Model(inputs=inputs3, outputs=outputs3)

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
his=model.fit(datagen.flow(trainx,trainy,batch_size=32),validation_data=(testx,testy),epochs=50)

In [ ]:
y_pred=model.predict(testx)
pred=np.argmax(y_pred,axis=1)
ground = np.argmax(testy,axis=1)
print(classification_report(ground,pred))

In [ ]:
get_acc = his.history['accuracy']
value_acc = his.history['val_accuracy']
get_loss = his.history['loss']
validation_loss = his.history['val_loss']

epochs = range(len(get_acc))
plt.plot(epochs, get_acc, 'r', label='Accuracy of Training data')
plt.plot(epochs, value_acc, 'b', label='Accuracy of Validation data')
plt.title('Training vs validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
epochs = range(len(get_loss))
plt.plot(epochs, get_loss, 'r', label='Loss of Training data')
plt.plot(epochs, validation_loss, 'b', label='Loss of Validation data')
plt.title('Training vs validation loss')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
pred2=model.predict(testX)
print(pred2.shape)

PRED=[]
for item in pred2:
    value2=np.argmax(item)      
    PRED+=[value2]
print(pd.Series(PRED).value_counts())

In [ ]:
ANS=testY0
print(pd.Series(ANS).value_counts())
accuracy=accuracy_score(ANS,PRED)
print(accuracy)

In [ ]:
fig, axs = plt.subplots(3,3,figsize=(12,12))
for i in range(9):
    r=i//3
    c=i%3
    img1 = testX[i]
    ax=axs[r][c].axis("off")
    actual=reverse_mapping[testY0[i]]
    predict=reverse_mapping[PRED[i]]    
    ax=axs[r][c].set_title(actual+'=='+predict)
    ax=axs[r][c].imshow(img1)
plt.show()